Please go through the "building_strategies" notebook first before you go through this notebook


## Some Prebuilt Reporting ##

Lets first build the strategy described in that notebook, add it to a portfolio and run the portfolio

In [1]:
from types import SimpleNamespace
import pandas as pd
import numpy as np
import pyqstrat as pq
from build_example_strategy import build_example_strategy
pq.__file__
strategy = build_example_strategy()
strategy.run()
# strategy = build_example_strategy(SimpleNamespace(lookback_period = 10, num_std = 2))
# strategy.run()

[2023-10-23 19:45:41.558 __call__] TRADE: 2023-01-13T09:32 AAPL  2023-01-13 09:32:00 qty: 151471 prc: 131.77   order: AAPL 2023-01-13 09:31:00 qty: 151471 POS_OVERNIGHT_RETURN  OrderStatus.OPEN 
[2023-10-23 19:45:41.559 __call__] ORDER: 2023-01-13T15:59 AAPL 2023-01-13 15:59:00 qty: -151471 EOD  OrderStatus.OPEN
[2023-10-23 19:45:41.559 __call__] TRADE: 2023-01-13T16:00 AAPL  2023-01-13 16:00:00 qty: -151471 prc: 134.74   order: AAPL 2023-01-13 15:59:00 qty: -151471 EOD  OrderStatus.OPEN 
[2023-10-23 19:45:41.561 __call__] TRADE: 2023-01-19T09:32 AAPL  2023-01-19 09:32:00 qty: 215649 prc: 134.77   order: AAPL 2023-01-19 09:31:00 qty: 215649 POS_OVERNIGHT_RETURN  OrderStatus.OPEN 
[2023-10-23 19:45:41.562 __call__] ORDER: 2023-01-19T10:56 AAPL 2023-01-19 10:56:00 qty: -215649 STOPPED_OUT  OrderStatus.OPEN
[2023-10-23 19:45:41.562 __call__] TRADE: 2023-01-19T10:57 AAPL  2023-01-19 10:57:00 qty: -215649 prc: 133.901   order: AAPL 2023-01-19 10:56:00 qty: -215649 STOPPED_OUT  OrderStatus.O

Many objects have functions that return pandas dataframes for ease of use.  Any function that returns a dataframe starts with df_ so its easy to tell which dataframes an object returns.

df_data is a useful function. This returns the market data, indicators, signal values and P&L at each market data bar.  The last column, i is the integer index of that bar, and can be used to query data in other dataframes or objects for that strategy.

Since we have daily pnl but minute bars a lot of the rows will have nans. So lets get EOD data (equity is not NaN in those rows)

In [2]:
df_data = strategy.df_data()
df_data[np.isfinite(df_data.equity)].head()

,timestamp,c,stop_price,eod_sig,overnight_ret_negative_sig,stop_sig,position,unrealized,realized,commission,fee,net_pnl,equity,i
389,2023-01-03 16:00:00,125.000000,NaN,True,False,False,0.0,0.0,0.0,0.0,0.0,0.0,1000000.0,389
779,2023-01-04 16:00:00,126.360000,125.797054,True,False,False,0.0,0.0,0.0,0.0,0.0,0.0,1000000.0,779
1169,2023-01-05 16:00:00,125.120094,125.922225,True,False,True,0.0,0.0,0.0,0.0,0.0,0.0,1000000.0,1169
1559,2023-01-06 16:00:00,129.459990,125.240650,True,False,False,0.0,0.0,0.0,0.0,0.0,0.0,1000000.0,1559
1949,2023-01-09 16:00:00,130.140000,130.563900,True,False,True,0.0,0.0,0.0,0.0,0.0,0.0,1000000.0,1949


You can also look at just the PNL or just the marketdata by themselves.

In [3]:
strategy.df_pnl().tail()

,timestamp,position,unrealized,realized,commission,fee,net_pnl,equity
15,2023-01-24 16:00:00,0.0,0.0,1.281979e+06,0.0,0.0,1.281979e+06,2.281979e+06
16,2023-01-25 16:00:00,0.0,0.0,1.092476e+06,0.0,0.0,1.092476e+06,2.092476e+06
17,2023-01-26 16:00:00,0.0,0.0,1.092476e+06,0.0,0.0,1.092476e+06,2.092476e+06
18,2023-01-27 16:00:00,0.0,0.0,1.583970e+06,0.0,0.0,1.583970e+06,2.583970e+06
19,2023-01-30 16:00:00,0.0,0.0,1.340368e+06,0.0,0.0,1.340368e+06,2.340368e+06


We can look at orders and trades that were created during this run

In [4]:
strategy.df_orders().head()

,symbol,type,timestamp,qty,reason_code,order_props,contract_props
0,AAPL,MarketOrder,2023-01-13 09:31:00,0,POS_OVERNIGHT_RETURN,,
1,AAPL,MarketOrder,2023-01-13 15:59:00,0,EOD,,
2,AAPL,MarketOrder,2023-01-19 09:31:00,0,POS_OVERNIGHT_RETURN,,
3,AAPL,MarketOrder,2023-01-19 10:56:00,0,STOPPED_OUT,,
4,AAPL,MarketOrder,2023-01-20 09:31:00,0,POS_OVERNIGHT_RETURN,,


In [5]:
strategy.df_trades().head()

,symbol,timestamp,qty,price,fee,commission,order_date,order_qty,reason_code,order_props,contract_props
0,AAPL,2023-01-13 09:32:00,151471,131.77000,0,0.0,2023-01-13 09:31:00,0,POS_OVERNIGHT_RETURN,,
1,AAPL,2023-01-13 16:00:00,-151471,134.73999,0,0.0,2023-01-13 15:59:00,0,EOD,,
2,AAPL,2023-01-19 09:32:00,215649,134.77000,0,0.0,2023-01-19 09:31:00,0,POS_OVERNIGHT_RETURN,,
3,AAPL,2023-01-19 10:57:00,-215649,133.90059,0,0.0,2023-01-19 10:56:00,0,STOPPED_OUT,,
4,AAPL,2023-01-20 09:32:00,187102,134.57501,0,0.0,2023-01-20 09:31:00,0,POS_OVERNIGHT_RETURN,,


In [6]:
strategy.df_roundtrip_trades()

,symbol,multiplier,entry_timestamp,exit_timestamp,qty,entry_price,exit_price,entry_reason,exit_reason,entry_commission,exit_commission,net_pnl
0,AAPL,1.0,2023-01-13 09:32:00,2023-01-13 16:00:00,151471,131.77000,134.73999,POS_OVERNIGHT_RETURN,EOD,0.0,0.0,449867.35529
1,AAPL,1.0,2023-01-19 09:32:00,2023-01-19 10:57:00,215649,134.77000,133.90059,POS_OVERNIGHT_RETURN,STOPPED_OUT,0.0,0.0,-187487.39709
2,AAPL,1.0,2023-01-20 09:32:00,2023-01-20 16:00:00,187102,134.57501,137.87001,POS_OVERNIGHT_RETURN,EOD,0.0,0.0,616501.09000
3,AAPL,1.0,2023-01-24 09:32:00,2023-01-24 16:00:00,267133,141.15102,142.66000,POS_OVERNIGHT_RETURN,EOD,0.0,0.0,403098.35434
4,AAPL,1.0,2023-01-25 09:32:00,2023-01-25 10:38:00,326673,139.56000,138.97990,POS_OVERNIGHT_RETURN,STOPPED_OUT,0.0,0.0,-189503.00730
5,AAPL,1.0,2023-01-27 09:32:00,2023-01-27 16:00:00,290823,144.23999,145.93000,POS_OVERNIGHT_RETURN,EOD,0.0,0.0,491493.77823
6,AAPL,1.0,2023-01-30 09:32:00,2023-01-30 09:43:00,355624,145.34500,144.66000,POS_OVERNIGHT_RETURN,STOPPED_OUT,0.0,0.0,-243602.44000


You can also look at the returns at the portfolio level (i.e. summing up several strategies)

In [7]:
strategy.df_returns().tail()

,timestamp,net_pnl,equity,ret
22,2023-01-24,1.281979e+06,2.281979e+06,0.214542
23,2023-01-25,1.092476e+06,2.092476e+06,-0.083043
24,2023-01-26,1.092476e+06,2.092476e+06,0.000000
25,2023-01-27,1.583970e+06,2.583970e+06,0.234886
28,2023-01-30,1.340368e+06,2.340368e+06,-0.094274


We can get data as native Python objects as opposed to pandas dataframes.

In [8]:
strategy.trades(start_date = np.datetime64('2023-01-13'), end_date = np.datetime64('2023-01-15'))

[AAPL  2023-01-13 09:32:00 qty: 151471 prc: 131.77   order: AAPL 2023-01-13 09:31:00 qty: 0 POS_OVERNIGHT_RETURN  OrderStatus.FILLED ,
 AAPL  2023-01-13 16:00:00 qty: -151471 prc: 134.74   order: AAPL 2023-01-13 15:59:00 qty: 0 EOD  OrderStatus.FILLED ]

## Adding your Own Metrics ##

Each strategy may have metrics that you want to measure that are specific to that strategy.  To add these, you can use the Evaluator object which can make things easier.

To evaluate a strategy we use the evaluate returns function.

In [9]:
strategy.evaluate_returns(plot = False);

,gmean,amean,std,shrp,srt,k,calmar,mar,mdd_pct,mdd_dates,dd_3y_pct,dd_3y_timestamps,up_dwn,2023
,7.904e+04,13.62,0.1606,5.341,23.01,6.003,105.3,105.3,0.1293,2023-01-18/2023-01-19,0.1293,2023-01-18/2023-01-19,4/3/0.571,7.904e+04


What if we want to add some more metrics to this.  For example, lets say we want to add a metric that looks at how many long trades we had versus short trades.  We can do this using an Evaluator object.

In [10]:
def compute_num_stopped_trades(trades):
    return len([trade for trade in trades if trade.order.reason_code == 'STOPPED_OUT'])

evaluator = pq.Evaluator(initial_metrics = {'trades' : strategy.trades()})

evaluator.add_metric('num_stopped_trades', compute_num_stopped_trades, dependencies = ['trades'])

evaluator.compute()

print(f'Stopped Tradess: {evaluator.metric("num_stopped_trades")}')

Stopped Tradess: 3


The Evaluator takes care of dependency management so that if you want to compute a metric that relies on other metrics, it will compute the metrics in the right order.

Lets compute Maximum Adverse Execution for each trade.  MAE tells you the maximum loss each trade had during its lifetime. It's useful for figuring out where to put trailing stops. For example, if most of your profitable trades had a maximum loss during their life up to 5% but many losing trades had losses of 50% and 60%, it might make sense to place a trailing stop around 6% or 7% so you don't get stopped out of your profitable trades but get out of the losing ones quickly. See Jaekle and Tomasini, page 66 for details

In [11]:
def compute_mae(rt_trades, c, timestamps):
    mae = np.full(len(rt_trades), np.nan)
    round_trip_pnl = np.full(len(rt_trades), np.nan)

    for i, rt in enumerate(rt_trades):
        _c = c[(timestamps >= rt.entry_timestamp) & (timestamps <= rt.exit_timestamp)]
        _mae = np.min(_c) / rt.entry_price - 1
        _mae = min(0, _mae)   # if we did not get a drawdown for this trade
        mae[i] = -_mae
        round_trip_pnl[i] = rt.net_pnl / rt.qty # Also store round trip pnl for this trade for plotting
    return mae, round_trip_pnl
        
contract_group = strategy.contract_groups[0]
evaluator = pq.Evaluator(initial_metrics = {'rt_trades' : strategy.roundtrip_trades(),
                                            'c' : strategy.indicator_values[contract_group].c,
                                            'timestamps' : strategy.timestamps})
evaluator.add_metric('mae', compute_mae, dependencies=['rt_trades', 'c', 'timestamps'])
evaluator.compute()

TypeError: unhashable type: 'ContractGroup'

We could have easily run the same computation without using the Evaluator.  The main advantage of using the Evaluator is that you can reuse other metrics you are dependent on without having to recompute them each time, i.e it provides a local cache of metrics.

In [ ]:
mae = evaluator.metric('mae')[0]
round_trip_pnl = evaluator.metric('mae')[1] 

# Separate out positive trades from negative trades
round_trip_profit = round_trip_pnl[round_trip_pnl >= 0]
mae_profit = mae[round_trip_pnl >= 0] 

round_trip_loss = round_trip_pnl[round_trip_pnl <= 0]
mae_loss = mae[round_trip_pnl <= 0]

import plotly.graph_objects as go
fig = go.Figure()
winners = go.Scatter(name='Profitable Trade', x=mae_profit, y=round_trip_profit, mode='markers', marker_color='green', marker_size=10, marker_symbol='triangle-up')
losers = go.Scatter(name='Losing Trade', x=mae_loss, y=-round_trip_loss, mode='markers', marker_color='red', marker_size=10, marker_symbol='triangle-down')
fig.add_trace(winners)
fig.add_trace(losers)
fig.add_hline(y=0, opacity=0.25)
fig.add_vline(x=0, opacity=0.25)
fig.update_xaxes(title_text='Drawdown %')
fig.update_yaxes(title_text='Profit / Loss %')
fig.show()

It looks like a good place to put a stop loss so we keep most of the winning trades but don't take big losses might be around 0.3%